In [5]:


def git_diff(repo_path, path, start_commit):
    
    repo = git.Repo(repo_path)
    # end_commit = repo.head.object.hexsha
    end_commit = 'a071d2e1880854899a9ab674770fcea35ea7344c'
    commit_a = repo.commit(start_commit)
    commit_b = repo.commit(end_commit)
    
    diff_output = ''
    command = ['git','diff',f'{commit_a}', f'{commit_b}']

    try:
        diff_output = subprocess.run(command, cwd=repo_path, capture_output=True, text=True, check=True).stdout
    except subprocess.CalledProcessError as e:
        print(f"Error executing git diff: {e}")
    print('testting', diff_output)
    diff_info = diff_output.split("\ndiff --git")
    diff_info = [part.strip() for part in diff_info if part.strip()]

    def git_changes(git_diff_output):
        changes={}
        diff_line = git_diff_output.splitlines()
        for line in diff_line:
            if line.startswith('diff --git') or line.startswith('a/'):
                start_index = line.find('a/') 
                end_index = line.find(' b/', start_index)
                file_name = line[start_index:end_index].split('/')[-1]
                chunk_headers = [line for line in diff_line if line.startswith("@@")]
                method_names = set()
                for header in chunk_headers:
                    parts = header.split("@@")[-1].strip().split("(")[0]
                    method_names.add(parts.split(' ')[-1])
                changes[file_name] = list(method_names)
        return changes

    final_changes = {}
    
    for ele in diff_info:
        changes = git_changes(ele)
        final_changes.update(changes)
    df_changes = pd.DataFrame([(file,method) for file, methods in final_changes.items() for method in methods], columns=['File_Name','Methods'])
    git_changes_path = os.path.join(path,"git_changes")
    if not os.path.exists(git_changes_path):
        os.makedirs(git_changes_path)
    with pd.ExcelWriter(os.path.join(git_changes_path,"changes.xlsx")) as w:
        df_changes.to_excel(w)
    
    return final_changes

import database
repo_path               = database.REPO_PATH
#executable_path         = database.EXECUTABLE_PATH
#test_scripts_path       = database.TEST_SCRIPTS_PATH
result_path             = database.RESULT_PATH
start_commit            = database.START_COMMIT
(git_diff(repo_path, result_path, start_commit))


testting diff --git a/.gitattributes b/.gitattributes
new file mode 100644
index 0000000..1d3953f
--- /dev/null
+++ b/.gitattributes
@@ -0,0 +1 @@
+*.cpp diff=cpp
\ No newline at end of file
diff --git a/.gitignore b/.gitignore
new file mode 100644
index 0000000..cec96f9
--- /dev/null
+++ b/.gitignore
@@ -0,0 +1,3 @@
+impactbasedtesting/impact/*
+textedit/build-textedit_v1-Desktop_Qt_6_6_2_MinGW_64_bit-Release/*
+impactbasedtesting/ScriptGeneration_202401191029/*
\ No newline at end of file
diff --git a/impactbasedtesting/.ipynb_checkpoints/dummy-checkpoint.ipynb b/impactbasedtesting/.ipynb_checkpoints/dummy-checkpoint.ipynb
new file mode 100644
index 0000000..58e6b34
--- /dev/null
+++ b/impactbasedtesting/.ipynb_checkpoints/dummy-checkpoint.ipynb
@@ -0,0 +1,147 @@
+{
+ "cells": [
+  {
+   "cell_type": "code",
+   "execution_count": 4,
+   "metadata": {},
+   "outputs": [
+    {
+     "ename": "ModuleNotFoundError",
+     "evalue": "No module named 'git'",
+     "output_type": "error",

{'.gitattributes': [''],
 '.gitignore': [''],
 'dummy-checkpoint.ipynb': [''],
 'database.cpython-312.pyc': [],
 'app.py': [''],
 'TC1StartApplication.csv': [''],
 'TC2SaveFile.csv': [''],
 'TC3RibbonTabs.csv': [''],
 'database.py': [''],
 'dummy.ipynb': [''],
 'generate_coverage_report.py': [''],
 'requirements.txt': [''],
 'coverage_test001.xlsx': [],
 'coverage_test002.xlsx': [],
 'coverage_test003.xlsx': [],
 'coverage_test_LaunchApplication.xlsx': [],
 'changes.xlsx': [],
 'suggest_test_cases.py': [''],
 'test.py': [''],
 '.qmake.stash': [],
 'QAction.D3763D8EE4FE922E.idx': [],
 'QActionGroup.1A3EB3D6290480A4.idx': [],
 'QApplication.2FFF2FE7AA837FD8.idx': [],
 'QClipboard.F7C87AF13668BD11.idx': [],
 'QCloseEvent.1BC14C86D2F0BDC7.idx': [],
 'QColorDialog.400F19AA1A20C7B1.idx': [],
 'QComboBox.3E5CE580A69ABADE.idx': [],
 'QFile.68697DFA13DB2A9A.idx': [],
 'QFileDialog.6D91814790C94595.idx': [],
 'QFileInfo.48ACA0DFD1F01A2F.idx': [],
 'QFontComboBox.C3F43487438352A0.idx': [],
 'QFon

In [1]:
import git
import subprocess
import os
import glob
import pandas as pd

In [1]:
!pip install GitPython